In [1]:
############################## can we decide between task related and MW ? ############################

# only two categorries

## Dependencies

# Import package/module for data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt


# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupKFold

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# cross validation
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold


#accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from IPython.display import display

from collections import Counter


In [2]:
read_path_awareness = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_label_awareness.csv"
read_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_labels.csv"
df = pd.read_csv(read_path)

#print(df.columns.tolist())

In [3]:
df["Awareness_all_cat"] =  df["Awareness_all_cat"].replace( 3,2)

In [4]:
# ['task-related' == 1 , 'aware' == 2, 'unaware' == 3 , nan == delete the row]
def get_X_y(train):
    FEATURES = [
        'Fixation Duration Mean [ms]', 'Fixation Duration Max [ms]', 'Fixation Duration Min [ms]', 'Fixation Duration Median [ms]', 'Fixation Duration Std [ms]', 'Fixation Duration Skew [ms]', 'Fixation Duration Quantil 25 [ms]', 'Fixation Duration Quantil 75 [ms]',
        'Saccade Duration Mean [ms]', 'Saccade Duration Max [ms]', 'Saccade Duration Min [ms]', 'Saccade Duration Median [ms]', 'Saccade Duration Std [ms]', 'Saccade Duration Skew [ms]', 'Saccade Duration Quantil 25 [ms]', 'Saccade Duration Quantil 75 [ms]', 
        'Blink Duration Mean [ms]', 'Blink Duration Max [ms]', 'Blink Duration Min [ms]', 'Blink Duration Median [ms]', 'Blink Duration Std [ms]', 'Blink Duration Skew [ms]', 'Blink Duration Quantil 25 [ms]', 'Blink Duration Quantil 75 [ms]', 'Fixation Duration Kurtosis [ms]',
        'Saccade Duration Kurtosis [ms]',
        'Blink Duration Kurtosis [ms]', 
        'Fixation Saccade Ratio Mean', 'Fixation Saccade Ratio Max', 'Fixation Saccade Ratio Min', 'Fixation Saccade Ratio Median', 'Fixation Saccade Ratio Std', 'Fixation Saccade Ratio Skew', 'Fixation Saccade Ratio Kurtosis', 
        'Fixation Number', 'Blink Number', 
        'Fixation Dispersion X Mean [px]', 'Fixation Dispersion X Max [px]', 'Fixation Dispersion X Min [px]', 'Fixation Dispersion X Median [px]', 'Fixation Dispersion X Std [px]', 'Fixation Dispersion X Skew [px]', 'Fixation Dispersion X Quantil 25 [px]', 'Fixation Dispersion X Quantil 75 [px]', 
        'Fixation Dispersion Y Mean [px]', 'Fixation Dispersion Y Max [px]', 'Fixation Dispersion Y Min [px]', 'Fixation Dispersion Y Median [px]', 'Fixation Dispersion Y Std [px]', 'Fixation Dispersion Y Skew [px]', 'Fixation Dispersion Y Quantil 25 [px]', 'Fixation Dispersion Y Quantil 75 [px]', 'Fixation Dispersion X Kurtosis [px]', 'Fixation Dispersion Y Kurtosis [px]', 
        'Saccade Amplitude Mean [°]', 'Saccade Amplitude Max [°]', 'Saccade Amplitude Min [°]', 'Saccade Amplitude Median [°]', 'Saccade Amplitude Std [°]', 'Saccade Amplitude Skew [°]', 'Saccade Amplitude Quantil 25 [°]', 'Saccade Amplitude Quantil 75 [°]', 'Saccade Amplitude Kurtosis [°]',
        'Saccade Acceleration Average [°/s²] Mean', 'Saccade Acceleration Average [°/s²] Max', 'Saccade Acceleration Average [°/s²] Min', 'Saccade Acceleration Average [°/s²] Median', 'Saccade Acceleration Average [°/s²] Std', 'Saccade Acceleration Average [°/s²] Skew]', 'Saccade Acceleration Average [°/s²] Quantil 25]', 'Saccade Acceleration Average [°/s²] Quantil 75]',
        'Saccade Acceleration Peak [°/s²] Mean', 'Saccade Acceleration Peak [°/s²] Max', 'Saccade Acceleration Peak [°/s²] Min', 'Saccade Acceleration Peak [°/s²] Median', 'Saccade Acceleration Peak [°/s²] Std', 'Saccade Acceleration Peak [°/s²] Skew]', 'Saccade Acceleration Peak [°/s²] Quantil 25]', 'Saccade Acceleration Peak [°/s²] Quantil 75]', 'Saccade Deceleration Peak [°/s²] Mean', 
        'Saccade Deceleration Peak [°/s²] Max', 'Saccade Deceleration Peak [°/s²] Min', 'Saccade Deceleration Peak [°/s²] Median', 'Saccade Deceleration Peak [°/s²] Std', 'Saccade Deceleration Peak [°/s²] Skew]', 'Saccade Deceleration Peak [°/s²] Quantil 25]', 'Saccade Deceleration Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Average [°/s²] Mean', 'Saccade Velocity Average [°/s²] Max', 'Saccade Velocity Average [°/s²] Min', 'Saccade Velocity Average [°/s²] Median', 'Saccade Velocity Average [°/s²] Std', 'Saccade Velocity Average [°/s²] Skew]', 'Saccade Velocity Average [°/s²] Quantil 25]', 'Saccade Velocity Average [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [°/s²] Mean', 'Saccade Velocity Peak [°/s²] Max', 'Saccade Velocity Peak [°/s²] Min', 'Saccade Velocity Peak [°/s²] Median', 'Saccade Velocity Peak [°/s²] Std', 'Saccade Velocity Peak [°/s²] Skew]', 'Saccade Velocity Peak [°/s²] Quantil 25]', 'Saccade Velocity Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [%] Mean', 'Saccade Velocity Peak [%] Max', 'Saccade Velocity Peak [%] Min', 'Saccade Velocity Peak [%] Median', 'Saccade Velocity Peak [%] Std', 'Saccade Velocity Peak [%] Skew]', 'Saccade Velocity Peak [%] Quantil 25]', 'Saccade Velocity Peak [%] Quantil 75]', 
        'Saccade Acceleration Average [°/s²] Kurtosis', 'Saccade Acceleration Peak [°/s²] Kurtosis', 'Saccade Deceleration Peak [°/s²] Kurtosis', 'Saccade Velocity Average [°/s²] Kurtosis', 'Saccade Velocity Peak [°/s²] Kurtosis', 'Saccade Velocity Peak [%] Kurtosis', 
        'Saccade Length Mean [px]', 'Saccade Length Max [px]', 'Saccade Length Min [px]', 'Saccade Length Median [px]', 'Saccade Length Std [px]', 'Saccade Length Skew [px]]', 'Saccade Length Quantil 25 [px]]', 'Saccade Length Quantil 75 [px]]', 'Saccade Length Kurtosis [px]', 
        'Fixation Average Pupil Diameter [mm] Mean', 'Fixation Average Pupil Diameter [mm] Max', 'Fixation Average Pupil Diameter [mm] Min', 'Fixation Average Pupil Diameter [mm] Median', 'Fixation Average Pupil Diameter [mm] Std', 'Fixation Average Pupil Diameter [mm] Skew', 'Fixation Average Pupil Diameter [mm] Quantil25', 'Fixation Average Pupil Diameter [mm] Quantil75',
        'Fixation Average Pupil Diameter [mm] Kurtosis', 
        # 'Veregence Angles Mean [rad]', 'Veregence Angles Std [rad]', 
        # 'Pupil Distance Mean [px]', 'Pupil Distance Std [px]'
    ]

    GROUPS = "Participant"

    TARGET = "Awareness_all_cat"

    X = train[FEATURES]
    y = train[TARGET]
    groups = train[GROUPS]
    return X, y, groups

In [5]:
# How many from each target category are in the set?
df2 = df.pivot_table(index = ['Awareness_all_cat'], aggfunc ='size')
print(df2)

Awareness_all_cat
1    631
2    396
dtype: int64


In [6]:
X, y, groups = get_X_y(df)

In [7]:
####### pipeline

# preprocessing with pipleline

# imputer 
# scaler
# model
pipe = Pipeline([
    ('imputer', SimpleImputer(fill_value='missing')),
    ('scaler',  StandardScaler()), # MinMaxScaler()), #
    ('model', SVC(kernel="rbf", C=0.025, probability=True)) #GaussianNB())## LinearSVC())
])

In [10]:
##################### With  Oversampling, Crossvalidation, imputing #################
## RandomOverSampler
## RandomOverSampler
from imblearn.over_sampling import RandomOverSampler
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

group_kfold = GroupKFold(n_splits=5)

fold = 0
accurancy_scores = []
f1_scores = []
### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    #print(f"Training target statistics: {Counter(y_train)}")
    #print(f"Testing target statistics: {Counter(y_test)}")
    over_sampler = RandomOverSampler(random_state=42)
    X_res, y_res = over_sampler.fit_resample(X_train, y_train)
    #print(f"Training target statistics with oversampling: {Counter(y_res)}")
    #print(f"Testing target statistics with oversampling: {Counter(y_test)}")

    # Fit Model on Train
    pipe.fit(X_res, y_res)
    y_pred = pipe.predict(X_test)
    y_baseline = [1] * len(y_pred)
    accurancy_score = accuracy_score(y_test, y_pred)
    #print(y_test.values)
    #print(y_pred)
    f1 = f1_score(y_test, y_pred)
    accurancy_scores.append(accurancy_score)
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    f1_scores.append(f1)
    print(f"Accuracy for fold {fold}: ", accurancy_score)
    print(f"f1  score for fold {fold}: ", f1)
    fold += 1

mean_score = np.mean(accurancy_scores)
mean_f1 = np.mean(f1_scores)
print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
print(f'Our mean fold f1 score is {mean_f1:0.4f}')


    0   1
0  77  53
1  33  42
Accuracy for fold 0:  0.5804878048780487
f1  score for fold 0:  0.6416666666666667
    0   1
0  98  55
1  19  38
Accuracy for fold 1:  0.6476190476190476
f1  score for fold 1:  0.7259259259259259
    0   1
0  57  54
1  31  66
Accuracy for fold 2:  0.5913461538461539
f1  score for fold 2:  0.5728643216080402
    0   1
0  82  41
1  57  29
Accuracy for fold 3:  0.5311004784688995
f1  score for fold 3:  0.6259541984732825
    0   1
0  76  38
1  39  42
Accuracy for fold 4:  0.6051282051282051
f1  score for fold 4:  0.6637554585152837
Our mean fold accurancy_scores is 0.5911
Our mean fold f1 score is 0.6460


In [9]:
##################### Oversampling #################
## RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=100)

group_kfold = GroupKFold(n_splits=5)

fold = 0
scores = []
### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    print(f"Training target statistics: {Counter(y_train)}")
    print(f"Testing target statistics: {Counter(y_test)}")
    over_sampler = RandomOverSampler(random_state=42)
    X_res, y_res = over_sampler.fit_resample(X_train, y_train)
    print(f"Training target statistics with oversampling: {Counter(y_res)}")
    print(f"Testing target statistics with oversampling: {Counter(y_test)}")

    # Fit Model on Train
    pipe.fit(X_res, y_res)
    y_pred = pipe.predict(X_test)
    y_baseline = [1] * len(y_pred)
    accurancy_score = accuracy_score(y_test, y_pred)
    scores.append(accurancy_score)
    print(f"Accuracy for fold {fold}: ", accurancy_score)
    fold += 1

mean_score = np.mean(scores)
print(f'Our mean fold score is {mean_score:0.4f}')


Training target statistics: Counter({1: 490, 2: 327})
Testing target statistics: Counter({1: 141, 2: 69})
Training target statistics with oversampling: Counter({1: 490, 2: 490})
Testing target statistics with oversampling: Counter({1: 141, 2: 69})
Accuracy for fold 0:  0.6333333333333333
Training target statistics: Counter({1: 488, 2: 330})
Testing target statistics: Counter({1: 143, 2: 66})
Training target statistics with oversampling: Counter({1: 488, 2: 488})
Testing target statistics with oversampling: Counter({1: 143, 2: 66})
Accuracy for fold 1:  0.5980861244019139
Training target statistics: Counter({1: 530, 2: 290})
Testing target statistics: Counter({2: 106, 1: 101})
Training target statistics with oversampling: Counter({1: 530, 2: 530})
Testing target statistics with oversampling: Counter({2: 106, 1: 101})
Accuracy for fold 2:  0.5458937198067633
Training target statistics: Counter({1: 512, 2: 324})
Testing target statistics: Counter({1: 119, 2: 72})
Training target statistic

In [ ]:
# f1 score: 


In [ ]:
# Chance: 0.5 
# our accuracy 0.5912
